# How to compute a functional map?

In [ ]:
import os

import numpy as np

from geomfun.descriptor.pipeline import (
    ArangeSubsampler,
    DescriptorPipeline,
    L2InnerNormalizer,
)
from geomfun.descriptor.spectral import HeatKernelSignature, WaveKernelSignature
from geomfun.functional_map import (
    FactorSum,
    LBCommutativityEnforcing,
    OperatorCommutativityEnforcing,
    SpectralDescriptorPreservation,
)
from geomfun.numerics.optimization import ScipyMinimize
from geomfun.shape import TriangleMesh

DATA_DIR = os.environ["GEOMFUN_DATA_DIR"]

[Load meshes](load_mesh_from_file.ipynb).

In [ ]:
filename_a = "cat-00.off"
filename_b = "lion-00.off"

mesh_a = TriangleMesh.from_file(f"{DATA_DIR}/{filename_a}")
mesh_b = TriangleMesh.from_file(f"{DATA_DIR}/{filename_b}")

[Set Laplace eigenbasis](./mesh_laplacian_spectrum.ipynb) for each mesh.

In [ ]:
mesh_a.find_laplacian_spectrum(spectrum_size=10, set_as_basis=True)
mesh_b.find_laplacian_spectrum(spectrum_size=10, set_as_basis=True)

mesh_b.basis.use_k = 8

Set a [descriptor pipeline](./descriptor_pipeline.ipynb) and apply it to both shapes.

In [ ]:
steps = [
    HeatKernelSignature(n_domain=4),
    ArangeSubsampler(subsample_step=2),
    WaveKernelSignature(n_domain=3),
    L2InnerNormalizer(),
]

pipeline = DescriptorPipeline(steps)

In [ ]:
descr_a = pipeline.apply(mesh_a)
descr_b = pipeline.apply(mesh_b)

Create objective function.

In [ ]:
factors = [
    SpectralDescriptorPreservation(
        mesh_a.basis.project(descr_a),
        mesh_b.basis.project(descr_b),
        weight=1.0,
    ),
    LBCommutativityEnforcing.from_bases(
        mesh_a.basis,
        mesh_b.basis,
        weight=1e-2,
    ),
    OperatorCommutativityEnforcing.from_multiplication(
        mesh_a.basis, descr_a, mesh_b.basis, descr_b, weight=1e-1
    ),
    OperatorCommutativityEnforcing.from_orientation(
        mesh_a, descr_a, mesh_b, descr_b, weight=1e-1
    ),
]

objective = FactorSum(factors)

Instantiate an `Optimizer` and solve for the functional map matrix.

In [ ]:
optimizer = ScipyMinimize(
    method="L-BFGS-B",
)

In [ ]:
x0 = np.zeros((mesh_a.basis.spectrum_size, mesh_b.basis.spectrum_size))

res = optimizer.minimize(
    objective,
    x0,
    fun_jac=objective.gradient,
)

fmap = res.x.reshape(x0.shape)

fmap.shape